In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns; sns.set()
import chart_studio.plotly as py
import cufflinks as cf
plt.style.use('ggplot')
%config InlineBackend.figure_format = 'retina'
cf.go_offline()

import matplotlib.font_manager as fm
path = 'C:\Windows\Fonts\malgunbd.ttf'
font_name = fm.FontProperties(fname=path).get_name()
plt.rc('font', family=font_name)

In [230]:
with open("Test.pickle",'rb') as f:
    Test = pickle.load(f)

In [231]:
Test['이용가능연령구분'] = Test["이용가능연령구분"].replace("Unrated","Everyone")

In [232]:
Divide = dict(list(Test.groupby('이용가능연령구분')))

In [233]:
Divide.keys()

dict_keys(['Everyone', 'Everyone 10+', 'Mature 17+', 'Teen'])

In [234]:
Everyone_Test = Divide['Everyone']

In [235]:
Teen_Test = Divide['Teen']

In [236]:
Everyone_10_Test = Divide['Everyone 10+']

In [237]:
Mature_17_Test = Divide['Mature 17+']

In [238]:
Everyone_Test  = Everyone_Test.drop(['카테고리','평가점수','유료구분','버전','안드로이드버전','이용가능연령구분'],axis = 1)

In [239]:
def remove(data):
    data  = data.drop(['카테고리','평가점수','유료구분','버전','안드로이드버전','이용가능연령구분'],axis = 1)
    return data

In [240]:
Teen_Test = remove(Teen_Test)

In [241]:
Everyone_10_Test = remove(Everyone_10_Test)

In [242]:
Mature_17_Test = remove(Mature_17_Test)

In [243]:
def Applen(data):
    data['app이름길이'] = data.app이름.apply(lambda x : len(x))
    data = data.drop(['app이름'],axis = 1)
    return data

In [244]:
def Appsize(data):
    data.app크기[data.app크기 == "Varies with device"] = "1M"
    Size = data.app크기.unique()
    
    app_size = []
    # M이 있을 경우에 제외하고 x1024, k있을 경우는 그냥 k만 지우자
    for i in Size:
        if i[-1] == "M":
            temp_data = float(i.replace("M",""))
            temp_data *= 1024
            app_size.append(temp_data)
        else:
            temp_data_2 = float(i.replace("k",""))
            app_size.append(temp_data_2)
            
    dict_app = {}
    for i in range(len(Size)):
        dict_app[Size[i]] = app_size[i]
    
    data['app크기'] = data.app크기.apply(lambda x : dict_app[x])
    
    return data

In [245]:
def Price_adh(data):
    data.가격 = data.가격.apply(lambda x : x.replace("$",""))
    data.가격 = data.가격.apply(lambda x : x.replace(",",""))
    data.가격 = data.가격.apply(lambda x : float(x.replace(",","")))
    return data

In [246]:
def Down_adj(data):
    data.다운로드수 = data.다운로드수.apply(lambda x : x.replace("+",""))
    data.다운로드수 = data.다운로드수.apply(lambda x : x.replace(",",""))
    data.다운로드수 = data.다운로드수.apply(lambda x : x.replace(",",""))
    data.다운로드수 = data.다운로드수.apply(lambda x : int(x))
    return data

In [247]:
def Version_adj(data):
    Month_dict = {"January" : 1, 'February' : 2, 'March' : 3, 'April' : 4, 'May' : 5,'June':6,'July' : 7,'August' : 8,'September' : 9,'October' : 10, "November" : 11, "December" : 12  }
    Month = data.마지막업데이트.str.split(' ').str[0]
    Month_data = Month.apply(lambda x : Month_dict[x])
    BBB = data.마지막업데이트.str.split(' ').str[1:]
    Day = BBB.str[0].apply(lambda x : int(x.replace(',','')))
    Year = BBB.str[1].apply(lambda x : int(x))
    return Month_data, Day, Year

In [248]:
def Update_version(data,Month,Day,Year):
    data = data.drop(['마지막업데이트'],axis = 1)
    data['업데이트_월'] = Month
    data['업데이트_일'] = Day
    data['업데이트_연도'] = Year
    return data

In [249]:
def All_adj(data):
    data = Applen(data)
    data = Appsize(data)
    data = Price_adh(data)
    data = Down_adj(data)
    Month,Day,Year = Version_adj(data)
    data = Update_version(data,Month,Day,Year)
    data.장르 = data.장르.str.split(';').str[0]  # 장르
    return data

In [259]:
Everyone_Test = All_adj(Everyone_Test)

In [165]:
Applen(Everyone_Test)

,app이름,리뷰수,app크기,다운로드수,가격,장르,마지막업데이트,app이름길이
23,Mcqueen Coloring pages,61,7168.0,100000,0,Art & Design;Action & Adventure,"March 7, 2018",22
123,Manicure - nail design,119,3788.8,50000,0,Beauty,"July 23, 2018",22
163,Anonymous caller detection,161,2764.8,10000,0,Books & Reference,"July 13, 2018",26
180,SH-02J Owner's Manual (Android 8.0),2,7372.8,50000,0,Books & Reference,"June 15, 2018",35
185,URBANO V 02 instruction manual,114,7475.2,100000,0,Books & Reference,"August 7, 2015",30
...,...,...,...,...,...,...,...,...
10823,List iptv FR,1,2969.6,100,0,Video Players & Editors,"April 22, 2018",12
10824,Cardio-FR,67,83968.0,10000,0,Medical,"July 31, 2018",9
10831,payermonstationnement.fr,38,10035.2,5000,0,Maps & Navigation,"June 13, 2018",24
10835,FR Forms,0,9830.4,10,0,Business,"September 29, 2016",8


In [166]:
Appsize(Everyone_Test)

C:\Anaconda\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



IndexError: invalid index to scalar variable.

In [167]:
Down_adj(Everyone_Test)

AttributeError: 'int' object has no attribute 'replace'

In [168]:
Month,Day,Year = Version_adj(Everyone_Test)

In [130]:
Month

23       3
123      7
163      7
180      6
185      8
        ..
10823    4
10824    7
10831    6
10835    9
10838    1
Name: 마지막업데이트, Length: 1295, dtype: int64

In [131]:
Everyone_Test = Update_version(Everyone_Test,Month,Day,Year)

In [132]:
Everyone_Test

,리뷰수,app크기,다운로드수,가격,장르,업데이트_월,업데이트_일,업데이트_연도
23,61,7168.0,100000,0,Art & Design;Action & Adventure,3,7,2018
123,119,3788.8,50000,0,Beauty,7,23,2018
163,161,2764.8,10000,0,Books & Reference,7,13,2018
180,2,7372.8,50000,0,Books & Reference,6,15,2018
185,114,7475.2,100000,0,Books & Reference,8,7,2015
...,...,...,...,...,...,...,...,...
10823,1,2969.6,100,0,Video Players & Editors,4,22,2018
10824,67,83968.0,10000,0,Medical,7,31,2018
10831,38,10035.2,5000,0,Maps & Navigation,6,13,2018
10835,0,9830.4,10,0,Business,9,29,2016


In [133]:
Price_adh(Everyone_Test)

,리뷰수,app크기,다운로드수,가격,장르,업데이트_월,업데이트_일,업데이트_연도
23,61,7168.0,100000,0.0,Art & Design;Action & Adventure,3,7,2018
123,119,3788.8,50000,0.0,Beauty,7,23,2018
163,161,2764.8,10000,0.0,Books & Reference,7,13,2018
180,2,7372.8,50000,0.0,Books & Reference,6,15,2018
185,114,7475.2,100000,0.0,Books & Reference,8,7,2015
...,...,...,...,...,...,...,...,...
10823,1,2969.6,100,0.0,Video Players & Editors,4,22,2018
10824,67,83968.0,10000,0.0,Medical,7,31,2018
10831,38,10035.2,5000,0.0,Maps & Navigation,6,13,2018
10835,0,9830.4,10,0.0,Business,9,29,2016


In [83]:
Everyone_Test.가격 = Everyone_Test.가격.apply(lambda x : x.replace("$",""))
Everyone_Test.가격 = Everyone_Test.가격.apply(lambda x : x.replace(",",""))
Everyone_Test.가격 = Everyone_Test.가격.apply(lambda x : float(x.replace(",","")))

In [86]:
Everyone_Test.가격.value_counts()

0.00      1152
0.99        38
1.49        14
1.99        13
2.99        13
5.99        12
9.99         5
2.49         5
3.99         4
6.49         4
5.49         3
4.99         2
2.00         2
24.99        2
399.99       1
1.04         1
1.96         1
14.99        1
8.99         1
89.99        1
1.26         1
3.61         1
200.00       1
1.00         1
109.99       1
4.80         1
39.99        1
19.90        1
2.60         1
30.99        1
394.99       1
28.99        1
29.99        1
19.99        1
154.99       1
25.99        1
8.49         1
46.99        1
74.99        1
4.85         1
Name: 가격, dtype: int64

In [134]:
Everyone_Test.장르 = Everyone_Test.장르.str.split(';').str[0]

In [135]:
Everyone_Test

,리뷰수,app크기,다운로드수,가격,장르,업데이트_월,업데이트_일,업데이트_연도
23,61,7168.0,100000,0.0,Art & Design,3,7,2018
123,119,3788.8,50000,0.0,Beauty,7,23,2018
163,161,2764.8,10000,0.0,Books & Reference,7,13,2018
180,2,7372.8,50000,0.0,Books & Reference,6,15,2018
185,114,7475.2,100000,0.0,Books & Reference,8,7,2015
...,...,...,...,...,...,...,...,...
10823,1,2969.6,100,0.0,Video Players & Editors,4,22,2018
10824,67,83968.0,10000,0.0,Medical,7,31,2018
10831,38,10035.2,5000,0.0,Maps & Navigation,6,13,2018
10835,0,9830.4,10,0.0,Business,9,29,2016


In [262]:
Teen_Test =  All_adj(Teen_Test)

In [263]:
Everyone_10_Test = All_adj(Everyone_10_Test)

In [264]:
Mature_17_Test = All_adj(Mature_17_Test)

In [265]:
with open("Everyone_Test.pickle",'wb') as f:
    pickle.dump(Everyone_Test,f)

In [266]:
with open("Teen_Test.pickle",'wb') as f:
    pickle.dump(Teen_Test,f)

In [267]:
with open("Everyone_10_Test.pickle",'wb') as f:
    pickle.dump(Everyone_10_Test,f)

In [268]:
with open("Mature_17_Test.pickle",'wb') as f:
    pickle.dump(Mature_17_Test,f)